In [ ]:
import socket
import threading
import FuncionesServidor as fs

# Manejo de conexiones de cliente
def handle_client(connection, client_address):
    error_message=''
    try:
        while True:  # Mantener la conexión abierta
            data = connection.recv(1024).decode()
            if not data:
                break  # Romper el bucle si no se recibe data
            else:
                command, username, password = data.split(' ')
            
            if command == 'exit':
                fs.remove_connection(username, client_address)
                break  # Romper el bucle si el comando es 'exit'
            elif command=='testconnection':
                connection.sendall('Usted sigue conectado al servidor\n'.encode())
            elif command == 'register':
                if fs.register_user(username, password):
                    connection.sendall('Registro exitoso\n'.encode())
                else:
                    error_message='Error: el usuario ya existe'
                    break
            elif command == 'login':
                if fs.login_user(username, password):
                    fs.add_connection(username, str(client_address))
                    connection.sendall('Login exitoso\n'.encode())
                else:
                    error_message='Error: usuario o contraseña incorrectos'
                    break
            else:
                break
    finally:
        connection.sendall(('Desconectado. Se le ha desconectado del servidor'+ error_message).encode())
        connection.close()
        

        
        
# Configuración del servidor
server_address = ('localhost', 6780)
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(server_address)
sock.listen()

print("Servidor iniciado, esperando conexiones...")

while True:
    connection, client_address = sock.accept()
    print(f"Conexión desde {client_address}")
    threading.Thread(target=handle_client, args=(connection, client_address)).start()

Servidor iniciado, esperando conexiones...
Conexión desde ('127.0.0.1', 63057)
